In [1]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [2]:
ranking_egresos_nacionales_deis = pl.read_csv(
    "../data/external/ranking_nacional_egresos.csv", encoding="latin-1", separator=";"
)

In [3]:
df_grd = pl.read_csv(
    "../data/processed/df_procesada.csv",
    separator=";",
    columns=["ESTANCIA", "COD_HOSPITAL", "DIAGNOSTICO1", "ANIO_EGRESO"],
)

df_grd = df_grd.with_columns(
    pl.col("DIAGNOSTICO1").str.replace(".", "", literal=True).str.pad_end(4, "X")
)

In [5]:
ranking_egresos_nacionales_grd = (
    df_grd.group_by(pl.col(["ANIO_EGRESO", "COD_HOSPITAL", "DIAGNOSTICO1"]))
    .agg([pl.count().alias("n_egresos"), pl.col("ESTANCIA").sum().alias("dias_estada_totales")])
    .sort(by=["ANIO_EGRESO", "DIAGNOSTICO1", "n_egresos"], descending=True)
    .with_columns(
        pl.col("n_egresos")
        .rank(method="min", descending=True)
        .over(["ANIO_EGRESO", "DIAGNOSTICO1"])
        .alias("ranking_egresos")
    )
)

ranking_egresos_nacionales_grd.write_csv(
    "../data/interim/ranking_nacionales_egresos_grd.csv", separator=";"
)

C:\Users\ppizarro\AppData\Local\Temp\ipykernel_16100\1508010986.py:3: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg([pl.count().alias("n_egresos"), pl.col("ESTANCIA").sum().alias("dias_estada_totales")])


In [6]:
comparacion_grd_deis_pais = ranking_egresos_nacionales_grd.join(
    ranking_egresos_nacionales_deis,
    left_on=["ANIO_EGRESO", "COD_HOSPITAL", "DIAGNOSTICO1"],
    right_on=["ANO_EGRESO", "ESTABLECIMIENTO_SALUD", "DIAG1"],
    how="inner",
    suffix="_DEIS",
)

In [7]:
COLUMNAS_A_SELECCIONAR = [
    "ANIO_EGRESO",
    "COD_HOSPITAL",
    "DIAGNOSTICO1",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "n_egresos",
    "n_egresos_DEIS",
    "ranking_egresos",
    "ranking_grd_n_egresos",
    "dias_estada_totales",
    "dias_estada_totales_DEIS",
    "ranking_nacionales_n_egresos",
    "ranking_publicos_n_egresos",
]

In [20]:
# Obtiene comparacion de egresos solamente para el Torax
comparacion_grd_deis_torax = (
    comparacion_grd_deis_pais.select(pl.col(COLUMNAS_A_SELECCIONAR))

    .filter(pl.col("COD_HOSPITAL") == 112103)
    .to_pandas()
)

# Calcula la diferencia de egresos entre GRD y DEIS para el Torax
comparacion_grd_deis_torax = abs(
    comparacion_grd_deis_torax["n_egresos"] - comparacion_grd_deis_torax["n_egresos_DEIS"]
)

# Obtiene el promedio de las diferencias de egresos
diferenca_promedio_egresos_torax = comparacion_grd_deis_torax.mean()
print(
    f"La diferencia promedio de egresos para el Torax es de: "
    f"{diferenca_promedio_egresos_torax:.2f} egresos"
)

La diferencia promedio de egresos para el Torax es de: 3.08 egresos
